# Машинное обучение, ФКН ВШЭ

## Практическое задание 3. Градиентный спуск своими руками

### Общая информация
Дата выдачи: 05.10.2019

Мягкий дедлайн: 07:59MSK 15.10.2019 (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 17.10.2019

### О задании

В данном задании необходимо реализовать обучение линейной регрессии с помощью различных вариантов градиентного спуска.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.


### Формат сдачи
Задания сдаются через систему Anytask. Инвайт можно найти на странице курса. Присылать необходимо ноутбук с выполненным заданием. 

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

**Оценка**: ...

## Реализация градиентного спуска

Реализуйте линейную регрессию с функцией потерь MSE, обучаемую с помощью:

** Задание 1 (1 балл)** Градиентного спуска;

** Задание 2 (1.5 балла)** Стохастического градиентного спуска;

** Задание 3 (2.5 балла)** Метода Momentum.


Во всех пунктах необходимо соблюдать следующие условия:

* Все вычисления должны быть векторизованы;
* Циклы средствами python допускается использовать только для итераций градиентного спуска;
* В качестве критерия останова необходимо использовать (одновременно):

    * проверку на евклидовую норму разности весов на двух соседних итерациях (например, меньше некоторого малого числа порядка $10^{-6}$, задаваемого параметром `tolerance`);
    * достижение максимального числа итераций (например, 10000, задаваемого параметром `max_iter`).
* Чтобы проследить, что оптимизационный процесс действительно сходится, будем использовать атрибут класса `loss_history` — в нём после вызова метода `fit` должны содержаться значения функции потерь для всех итераций, начиная с первой (до совершения первого шага по антиградиенту);
* Инициализировать веса можно случайным образом или нулевым вектором. 


Ниже приведён шаблон класса, который должен содержать код реализации каждого из методов.

In [75]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearReg(BaseEstimator):
    def __init__(self, gd_type='stochastic', 
                 tolerance=1e-7, max_iter=5000, w0=None, alpha=1e-3, eta=1e-3):
        """
        gd_type: 'full' or 'stochastic' or 'momentum'
        tolerance: for stopping gradient descent
        max_iter: maximum number of steps in gradient descent
        w0: np.array of shape (d) - init weights
        eta: learning rate
        alpha: momentum coefficient
        """
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.loss_history = None # list of loss function values at each training iteration
    
    def fit(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: self
        """
        self.loss_history = []
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        if self.w0 == None:
            self.w0 = np.zeros(X.shape[1])
        self.w = self.w0
        self.loss_history.append(self.calc_loss(X, y))
        if self.gd_type == 'full':
            for i in range(self.max_iter):
                #gradient step and new weights
                grad_step = self.eta*self.calc_gradient(X, y)
                self.w = self.w - grad_step
                #append loss history
                self.loss_history.append(self.calc_loss(X, y))
                #checking tolerance for weights
                if (grad_step**2).sum() < self.tolerance:
                    break
            
        return self
    
    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        return X.dot(self.w)
        pass
    
    def calc_gradient(self, X, y):
        """
        X: np.array of shape (ell, d) (ell can be equal to 1 if stochastic)
        y: np.array of shape (ell)
        ---
        output: np.array of shape (d)
        """
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        A = X.dot(self.w) - y
        grad = X.T.dot(A)*2/X.shape[0]
        return grad
        pass

    def calc_loss(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: float 
        """ 
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        return ((X.dot(self.w) - y)**2).sum()/X.shape[0]
        pass

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

** Задание 4 (0 баллов)**. 
* Загрузите данные из домашнего задания 2 ([train.csv](https://www.kaggle.com/c/nyc-taxi-trip-duration/data));
* Разбейте выборку на обучающую и тестовую в отношении 7:3 с random_seed=0;
* Преобразуйте целевую переменную `trip_duration` как $\hat{y} = \log{(y + 1)}$.

In [36]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
import pandas as pd
import math
df_train = pd.read_csv('train.csv')
df_train['log_trip_duration'] = np.log(df_train['trip_duration'] + 1)

#Data cleaning
df_train.drop('id',inplace=True,axis=1)
df_train.drop('dropoff_datetime',inplace=True,axis=1)
df_train["pickup_datetime"]=pd.to_datetime(df_train["pickup_datetime"])
df_train['day_of_week'] = df_train['pickup_datetime'].dt.day_name()
df_train['hour_of_the_day']=df_train['pickup_datetime'].dt.hour
df_train['month']=df_train['pickup_datetime'].dt.month
df_train['day_of_week']=df_train['day_of_week'].map({'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7})
df_train['store_and_fwd_flag']=df_train['store_and_fwd_flag'].map({'N':0,'Y':1})
def haversine(lat1, lon1, lat2, lon2):
    # distance between latitudes and longitudes
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
 
    # convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
 
    # apply formulae
    a = (pow(math.sin(dLat / 2), 2) +
         pow(math.sin(dLon / 2), 2) *
             math.cos(lat1) * math.cos(lat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    return rad * c
df_train['distance']=df_train.apply(lambda row:haversine(row['pickup_latitude'],row['pickup_longitude'],row['dropoff_latitude'],row['dropoff_longitude']),axis=1)
df_train['distance']=df_train['distance'].astype(float)
df_train.drop('pickup_longitude',inplace=True,axis=1)
df_train.drop('pickup_latitude',inplace=True,axis=1)
df_train.drop('dropoff_longitude',inplace=True,axis=1)
df_train.drop('dropoff_latitude',inplace=True,axis=1)
df_train.drop('pickup_datetime',inplace=True,axis=1)

#train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train.drop(columns = ['log_trip_duration', 'trip_duration']),
                                                            df_train['log_trip_duration'], test_size=0.3, random_state=0)
"""
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
"""

'\nfrom sklearn.preprocessing import StandardScaler\nscaler = StandardScaler()\nscaler.fit(X_train)\nX_train = scaler.transform(X_train)\nX_test = scaler.transform(X_test)\n'

In [76]:
reg = LinearReg(gd_type = 'full')
reg.fit(X_train, y_train)
reg.predict(X_test)

571578     6.079971
1280332    9.631096
177838     5.973925
1433776    6.783219
757662     4.950731
             ...   
1042944    9.106721
85930      3.485629
61268      9.060404
251164     7.441664
1348190    5.062943
Length: 437594, dtype: float64

In [77]:
reg.loss_history[-10:]

[1.8948393371265615,
 1.8947384327650874,
 1.8946376561764315,
 1.8945370071983676,
 1.8944364856688756,
 1.8943360914261427,
 1.8942358243085646,
 1.8941356841547443,
 1.8940356708034902,
 1.8939357840938191]

In [67]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
((reg.predict(X_train) - y_train)**2).sum()/X_train.shape[0]

0.41452899996385256

In [55]:
reg.loss_history

[42.45540471238833,
 38.64401536143907,
 35.224830961978626,
 32.15733964317357,
 29.40521500209197,
 26.935883677692118,
 24.72013760140298,
 22.731786308483194,
 20.9473451712318,
 19.345755842743614,
 17.90813558333912,
 16.617552485622053,
 15.45882392241763,
 14.418335818292851,
 13.483880593246166,
 12.64451184942976,
 11.890414071078482,
 11.212785786538827,
 10.603734801545162,
 10.05618425658858,
 9.563788390074578,
 9.120857004504693,
 8.72228773651926,
 8.363505324536117,
 8.040407151020768,
 7.749314411116982,
 7.4869283263440485,
 7.2502908821236876,
 7.036749621751972,
 6.843926077719976,
 6.669687464586187,
 6.512121296429796,
 6.369512626728346,
 6.2403236397209625,
 6.123175350310776,
 6.016831194660236,
 5.920182316140448,
 5.832234371476822,
 5.752095700030398,
 5.678966715380832,
 5.6121303929276065,
 5.550943740272994,
 5.494830148849694,
 5.4432725357462255,
 5.395807194089961,
 5.352018278782351,
 5.311532861944209,
 5.27401649921083,
 5.239169254097896,
 5.20672

** Задание 5 (3 балла)**. Обучите и провалидируйте модели на данных из предыдущего пункта, сравните качество между методами по метрикам MSE и $R^2$. Исследуйте влияние параметров `max_iter` и `eta` (`max_iter`, `alpha` и `eta` для Momentum) на процесс оптимизации. Согласуется ли оно с вашими ожиданиями?

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

** Задание 6 (2 балла)**. Постройте графики (на одной и той же картинке) зависимости величины функции потерь от номера итерации для полного, стохастического градиентного спусков, а также для полного градиентного спуска с методом Momentum. Сделайте выводы о скорости сходимости различных модификаций градиентного спуска.

Не забывайте о том, что должны получиться *красивые* графики!

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

### Бонус 

** Задание 7 (2 балла)**. Реализуйте линейную регрессию с функцией потерь MSE, обучаемую с помощью метода
[Adam](https://arxiv.org/pdf/1412.6980.pdf) - добавьте при необходимости параметры в класс модели, повторите пункты 5 и 6 и сравните результаты. 

** Задание 8 (2 балла)**. Реализуйте линейную регрессию с функцией потерь
$$ L(\hat{y}, y) = log(cosh(\hat{y} - y)),$$

обучаемую с помощью градиентного спуска.

** Задание 9 (0.01 балла)**.  Вставьте картинку с вашим любимым мемом в этот Jupyter Notebook